<a href="https://colab.research.google.com/github/OleKrarup123/PythonTutorials/blob/main/dual_y_axes/dual_y_axes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import useful libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load list of dataframes from webpage

In [ ]:
# Original data source: https://www.measuringworth.com/datasets/gold/

df_list=pd.read_html('https://github.com/OleKrarup123/PythonTutorials/blob/main/dual_y_axes/GOLD_1970-2020.csv')
df=df_list[0]    #Get pandas dataframe from first (and only) element in this list    
df.head() #Print head

## Add new column with price difference

In [ ]:
df['Gold price difference (NYC-LONDON) [$/ounce]'] = df['New York Price [$/ounce]'] -  df['London Market Price [$/ounce]']
df.head()

## Plot data separately

In [ ]:

plt.figure()
plt.title('Gold price in NYC and London')
plt.plot(df['Year'],df['New York Price [$/ounce]'])
plt.plot(df['Year'],df['London Market Price [$/ounce]'])
plt.xlabel('Year')
plt.ylabel('Gold price [$/ounce]')
plt.show()

plt.figure()
plt.title('Gold price difference')
plt.plot(df['Year'],df['Gold price difference (NYC-LONDON) [$/ounce]'])
plt.xlabel('Year')
plt.ylabel('Gold price difference (NYC - LONDON) [$/ounce]')
plt.show()

plt.figure()
plt.title('Gold/Silver price ratio')
plt.plot(df['Year'],df['Gold/Silver Price Ratio'])
plt.xlabel('Year')
plt.ylabel('Gold/Silver Price Ratio')
plt.show()


## Plot price difference and gold/silver ratio together

In [ ]:
#Initialize plot and graph price difference data
fig, ax1 = plt.subplots() 
ax1.plot(df['Year'], df['Gold price difference (NYC-LONDON) [$/ounce]'], color = 'C0') 
ax1.set_xlabel('Year') 
ax1.set_ylabel('Gold price difference (NYC - LONDON) [$/ounce]', color = 'C0') 
ax1.tick_params(axis ='y', labelcolor = 'C0') 

ax2=ax1.twinx() #Create new graph that shares the previous x-axis
ax2.plot(df['Year'], df['Gold/Silver Price Ratio'], color = 'C1') 
ax2.set_ylabel('Gold/Silver Price Ratio', color = 'C1') 
ax2.tick_params(axis ='y', labelcolor = 'C1') 

ax1.set_title('Gold price difference and Gold/Silver ratio')
fig.show()



 ...

...

...

## Create rolling average of Gold/Silver price ratio

In [ ]:
df['Gold/Silver Price Ratio 3 year roll-avg'] =df['Gold/Silver Price Ratio'].rolling(3).mean()
df['Gold/Silver Price Ratio 6 year roll-avg'] =df['Gold/Silver Price Ratio'].rolling(6).mean()

df.head()

In [ ]:

plt.figure()
plt.title('Gold/Silver Ratio')
plt.plot(df['Year'],df['Gold/Silver Price Ratio'], color='C1', label ='Raw data')
plt.plot(df['Year'],df['Gold/Silver Price Ratio 3 year roll-avg'], color='C2', label ='3yr roll-avg')
plt.plot(df['Year'],df['Gold/Silver Price Ratio 6 year roll-avg'], color='C3', label ='6yr roll-avg')
plt.xlabel('Year')
plt.ylabel('Gold/Silver Ratio')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()

...

...

...

## Compute "forward" derivative

In [ ]:
#Compute "backwards" derivative
df['NYC Gold price diff [$/ounce/year]'] = df['New York Price [$/ounce]'].diff()/df['Year'].diff()   
#dy/dx [0] = NaN
#dy/dx [1] = (y[1]-y[0])/(x[1]-x[0])
#dy/dx [2] = (y[2]-y[1])/(x[2]-x[1])
#...

#Shift entries backwards by 1 to get "forward derivative"
df['NYC Gold price diff [$/ounce/year]']=df['NYC Gold price diff [$/ounce/year]'].shift(periods=-1)
#dy/dx [0] = (y[1]-y[0])/(x[1]-x[0])
#dy/dx [1] = (y[2]-y[1])/(x[2]-x[1])
#dy/dx [2] = (y[3]-y[2])/(x[3]-x[2])
#...
#dy/dx [end] = NaN


df.head()


In [ ]:
df.tail()

## Plot derivative

In [ ]:
plt.figure()
plt.title('NYC Gold price')
plt.plot(df['Year'],df['New York Price [$/ounce]'], color='C0')
plt.xlabel('Year')
plt.ylabel('NYC Gold price [$/ounce]')
plt.show()

plt.figure()
plt.title('NYC Gold price change')
plt.plot(df['Year'],df['NYC Gold price diff [$/ounce/year]'], color='C4')
plt.xlabel('Year')
plt.ylabel('NYC Gold price diff [$/ounce/year]')
plt.show()

## Color gold price chart according to derivative

In [ ]:
# https://matplotlib.org/stable/gallery/lines_bars_and_markers/multicolored_line.html

from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap

#Make numpy array of points:
#
#  points = [[t0, y0] ,  [t1, y1], ... ]  
#
points = np.array([df['Year'], df['New York Price [$/ounce]']]).T.reshape(-1, 1, 2)

#Make numpy array of line segments:
#
#  segments = [   [[t0, y0], [t1, y1]]    ,    [[t1, y1], [t2, y2]]    , ... ]  
#
segments = np.concatenate([points[0:-1], points[1:]], axis=1)


# Create a continuous norm to map from derivative to line colour
norm = plt.Normalize(-200, 200)

#Turn segments into a collection of lines with a colour map and a colour normalization function
lc = LineCollection(segments, cmap='RdYlGn', norm=norm)

# Activate colour normalization function by selecting colour value based on derivative
lc.set_array(df['NYC Gold price diff [$/ounce/year]'])

#Set linewidth
lc.set_linewidth(5)

#Plot
fig, ax = plt.subplots()
line = ax.add_collection(lc)                                     #Add line collection to plot
fig.colorbar(line, ax=ax,label='Price change [$/ounce/year]')    #Add colorbar

#Specify axes
ax.set_xlim(df['Year'].min(), df['Year'].max())                   
ax.set_ylim(df['New York Price [$/ounce]'].min(), df['New York Price [$/ounce]'].max())

#Specify labels
ax.set_xlabel('Year')
ax.set_ylabel('New York Price [$/ounce]')
plt.show()